In [102]:
import torch
import pathlib
import os
import sounddevice as sd
import soundfile as sf
import pathlib
import torchaudio
import tempfile
import queue
from collections import namedtuple

In [17]:
# to successfully install pyaudio on linux
'''
sudo apt-get install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
'''
None

In [18]:
def resample(aud, newsr):
    sig, sr = aud

    if sr == newsr:
        # Nothing to do
        return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1, :])
    if num_channels > 1:
        # Resample the second channel and merge both channels
        retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:, :])
        resig = torch.cat([resig, retwo])

    return (resig, newsr)


In [21]:
base_path = "/home/lux_t1/Desktop/large-lightning-template-test"
os.chdir(base_path)
pathlib.Path.cwd()

PosixPath('/home/lux_t1/Desktop/large-lightning-template-test')

In [22]:
from src.models.audio_model_min import AudioLitModel

In [23]:
wd = pathlib.Path("/home/lux_t1/Desktop/large-lightning-template-test/notebooks")
os.chdir(wd)
pathlib.Path.cwd()

PosixPath('/home/lux_t1/Desktop/large-lightning-template-test/notebooks')

In [199]:
audiofile_path = pathlib.Path("test_data/soundscape_unimodal4.wav")

In [200]:
sample_rate = 41000
sd.default.samplerate = sample_rate

In [201]:
sig, sr = torchaudio.load(audiofile_path)
if sr is not sample_rate:
    sig, sr = resample((sig, sr), sample_rate)
sig = sig.reshape(1, *sig.size())
pred = trained_model(sig)
pred = torch.nn.Sigmoid()(pred)

pred[pred>0.5] = 1
pred[pred<=0.5] = 0

pred

tensor([[1., 1., 0., 1.]])

In [ ]:
# Change device
# sd.default.device = 12

In [255]:
def preprocess(data):
    data = data.swapaxes(0,1)
    return data

In [128]:
audiostream_tuple = namedtuple("audiostream_tuple", ["data", "frames"])

In [143]:
def callback(indata, frames, time, status):
    """This is called (from a separate thread) for each audio block."""
    if status:
        print(status, file=sys.stderr)
    q.put(audiostream_tuple(preprocess(indata.copy()), frames))

In [270]:
def predict(array_list, model):
    classes = ["Bell", "Cat", "Dishes", "Dog"]
    
    array_list = [_[np.newaxis, ...] for _ in array_list]
    batch = torch.Tensor(np.concatenate(array_list, axis = 0))
    batch = batch / batch.max()
    pred = model(batch)
    pred = torch.sigmoid(pred).numpy()
    pred = np.mean(pred, axis = 0)
    pred[pred>0.5] = True
    pred[pred<=0.5] = False
    print([_ for i,_ in enumerate(classes) if pred[i] == 1])
    return pred

In [245]:
from scipy.io.wavfile import write

In [282]:
DURATION = 2
CHANNELS = 1            # channels of recordings
RATE = 41000            # sample rate
REC_BUFFER_SIZE = 1024  # recorder buffer size, every records has channel's number of samples, if channel = 2, each record has 2 samples
                        # this parameter also determines the delay between recording and playback, LEN_REC_BUFFER / RATE (s) is the actual delay
chunk_len = DURATION * RATE // REC_BUFFER_SIZE          # 62 * REC_BUFFER_SIZE / RATE, approximately 2 (s) data for inference        (*algorithm cost*)
chunk_hs =  30          # 7 * REC_BUFFER_SIZE / RATE, approximately 1280 (ms), chunk level hop size   (*algorithm cost*)

classes = ["Cat", "Dog", "Dishes", "Alarm_bell_ringing", "Electric_shaver_toothbrush"]

In [280]:
trained_model = AudioLitModel.load_from_checkpoint(checkpoint_path="test.ckpt")
trained_model.eval()
trained_model.freeze()

/home/lux_t1/miniconda3/envs/torch/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The `multilabel=True` parameter is unused and will not have any effect.
  warnings.warn(*args, **kwargs)


In [283]:
class DataRecorded():

    def __init__(self):
        

        self.chunk = np.zeros((chunk_len, REC_BUFFER_SIZE))
        self.chunk_idx = 0
        self.chunk_hs_count = 0
        self.Flag_Chunk_Full = False

        self.chunk_ready = np.zeros((chunk_len, REC_BUFFER_SIZE))

In [296]:
class AudioDetection():
    
    def __init__(self, model, classes, cuda):

        self.model = model
        self.classes = classes
        self.cuda = cuda
        self.num_results = len(classes)
        if cuda:
            self.model.cuda()

        # initialize data format
        self.data = DataRecorded()

        # set record stream
        self.p = pyaudio.PyAudio()

        # start recording stream
        self.stream = self.p.open(format=pyaudio.paFloat32,
                channels=CHANNELS,
                rate=RATE,
                input=True, # record
                output=False, # playback
                frames_per_buffer=REC_BUFFER_SIZE,
                stream_callback=self.callback,
                start=False)
        
        self.running = True
        

    def callback(self, in_data, frame_count, time_info, status):
        '''
        Callback method is to unblockingly record and inference.
        Realtime signal processing could be added into this block.
        Segmentation of the time domain signal is done here.
        '''
        self.data.chunk[self.data.chunk_idx, :] = np.fromstring(in_data, dtype=np.float32)
        self.data.chunk_idx += 1
        if self.data.chunk_idx == chunk_len:
            self.data.chunk_idx = 0

        self.data.chunk_hs_count += 1
        if self.data.chunk_hs_count == chunk_hs:
            self.data.chunk_hs_count = 0
            self.data.Flag_Chunk_Full = True
            
        return (in_data, pyaudio.paContinue)
    

    def inference(self, x):
        '''
        Inference output for single instance from neural network
        '''
        print(x.shape)
#         x = torch.Tensor(x).view(1, x.shape[0], x.shape[1])
#         if cuda:
#             x = x.cuda()

#         prob = y.data.cpu().numpy().squeeze(axis=0)
#         return predict_idxs, predict_probs
    

    def start_detection(self, resultLabel, posthocProb):

        self.stream.start_stream()
        
        while self.running and self.stream.is_active():

            if self.data.Flag_Chunk_Full:

                self.data.chunk_ready = \
                    np.vstack((self.data.chunk[self.data.chunk_idx:,:], self.data.chunk[0:self.data.chunk_idx,:])).reshape(-1,)
                x = self.data.chunk_ready
                print(x.shape)

#                 # obtain labels and probabilities
#                 predict_idxs, predict_probs = self.inference(x)
#                 predict_labels = []
#                 for idx in predict_idxs:
#                     predict_labels.append(self.ix_to_lb[idx])
                
#                 # set GUI display
#                 for n in range(self.num_results):
#                     posthocProb[n].set(predict_probs[n])
#                     tempStr='{0:<15.15}'.format(predict_labels[n])
#                     resultLabel[n].set(tempStr)
    def stop_detection(self):

        self.stream.stop_stream()
        
    
    def terminate_detection(self):

        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()

<ipython-input-293-72b10c2e2e4e>:37: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  self.data.chunk[self.data.chunk_idx, :] = np.fromstring(in_data, dtype=np.float32)


In [297]:
detect = AudioDetection(trained_model, classes, True)

In [298]:
import pyaudio

In [299]:
detect.start_detection(None, None)

<ipython-input-288-9ddfce2adbd9>:37: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  self.data.chunk[self.data.chunk_idx, :] = np.fromstring(in_data, dtype=np.float32)


(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(81920,)
(

KeyboardInterrupt: 

In [276]:
batchsize_predict = 10
target_frames = int(samplerate * duration)
arrays = [np.zeros((channels, target_frames)) for _ in range(batchsize_predict)]
datalist = {_: [] for _ in range(batchsize_predict)}
frames = [0 for _ in range(batchsize_predict)]
recording = [False for _ in range(batchsize_predict)]

sample_difference = target_frames // batchsize_predict
samplerate = 41000
filename = "tmp.wav"
channels = 1
device = 12
q = queue.Queue()
audio_tuples = []
warmup = True
new_information = False

# try:
if samplerate is None:
    device_info = sd.query_devices(args.device, 'input')
    # soundfile expects an int, sounddevice provides a float:
    samplerate = int(device_info['default_samplerate'])

with sd.InputStream(samplerate=samplerate, device=device,
                    channels=channels, callback=callback):

    print('#' * 80)
    print('press Ctrl+C to stop the recording')
    print('#' * 80)


    recording[i] = True
    sum_frames = 0
    print("warmup")
    while warmup:
        audio_tuple = q.get()
        sum_frames += audio_tuple.frames
        n_recording = sum_frames // sample_difference
        recording[n_recording] = True
        
        for i in range(batchsize_predict):
            if recording[i] is True:
                if (frames[i] + audio_tuple.frames) < target_frames:
                    frames[i] += audio_tuple.frames
                    datalist[i].append(audio_tuple.data)
                    
        if n_recording == (batchsize_predict - 1):
            warmup = False
            
    print("running")
    while True:    
        audio_tuple = q.get()
        for i in range(batchsize_predict):
            if (frames[i] + audio_tuple.frames) < target_frames:
                    frames[i] += audio_tuple.frames
                    datalist[i].append(audio_tuple.data)
            else:
                new_information = True
                tmp_array = np.concatenate(datalist[i], axis = 1)
                dims = tmp_array.shape
                arrays[i][...,:dims[1]] = tmp_array.copy()[...,:]
                write(f"{i}.wav", samplerate, arrays[i][0])
                datalist[i] = [audio_tuple.data]
                frames[i] = audio_tuple.frames
        if new_information is True:
            pred = predict(arrays.copy(), trained_model)
            print(pred)
            new_information = False
            
# except KeyboardInterrupt:
#     print('\nRecording finished: ' + repr(filename))
#     parser.exit(0)
# except Exception as e:
#     print(e)